In [1]:
import pandas as pd

In [3]:
# 1.א
import openpyxl
import re

def read_xlsx_in_chunks(file_path, chunk_size=10000):
    """
    קורא קובץ XLSX בחלקים באמצעות גנרטור.
    
    """
    workbook = openpyxl.load_workbook(file_path, read_only=True)
    sheet = workbook.active
    chunk = []
    for row in sheet.rows:
        chunk.append(row[0].value)  
        if len(chunk) == chunk_size:
            yield chunk
            chunk = []
    if chunk:
        yield chunk
    workbook.close()

In [4]:
def read_log_file_in_chunks(file_path, chunk_size=10000):
    """
    קורא קובץ לוג txt בחלקים באמצעות גנרטור.

    """
    with open(file_path, 'r') as f:
        chunk = []
        for line in f:
            chunk.append(line.strip())  
            if len(chunk) == chunk_size:
                yield chunk
                chunk = []
        if chunk:
            yield chunk

In [5]:
def extract_error_codes(log_line):
    """
    מחפש אתהשגיאה באצעות regex.
    """
    match = re.search(r'Error: (\w+_\d+)', log_line)
    return match.group(1) if match else None

In [6]:
from collections import Counter

def process_chunk(chunk, results):
    """
    מעבד חלק של נתונים וסופר את קודי השגיאה.
    """
    error_counts = Counter()
    for log_line in chunk:
        error_code = extract_error_codes(log_line)
        if error_code:
            error_counts[error_code] += 1
    results.append(error_counts)

In [ ]:
import threading

def count_errors_parallel(file_path, num_threads=4):
    """
    סופר קודי שגיאות במקביל באמצעות threads.
    """
    results = []
    threads = []
    for chunk in read_xlsx_in_chunks(file_path):
    # for chunk in read_log_file_in_chunks(file_path): #--- עבור קובץ txt
        thread = threading.Thread(target=process_chunk, args=(chunk, results))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    merged_counts = Counter()
    for error_counts in results:
        merged_counts.update(error_counts)
    return merged_counts

In [ ]:
import heapq

def find_top_n_errors(error_counts, n):
    """
    מוצא את N קודי השגיאה השכיחים ביותר.
    :param error_counts: מילון ספירות שגיאות.
    :param n: מספר קודי השגיאה השכיחים ביותר למצוא.
    :return: רשימה של N קודי השגיאה השכיחים ביותר וספירותיהם.
    """
    return heapq.nlargest(n, error_counts.items(), key=lambda item: item[1])

In [ ]:
file_path = './logs.txt.xlsx' 
n = 10 

error_counts = count_errors_parallel(file_path)
top_n_errors = find_top_n_errors(error_counts, n)

print(f'Top {n} error codes:')
for error_code, count in top_n_errors:
    print(f'{error_code}: {count}')

Top 10 error codes:
WARN_101: 200098
ERR_404: 200094
ERR_400: 200069
INFO_200: 199931
ERR_500: 199808


In [10]:
# 1. סיבוכיות זמן:

# O(n + n log k) 
# O(n): קריאת הקובץ - בסוף הרי עוברים על כל השורות. 
# O(n log k): מיון וחיפוש N השגיאות הנפוצות ביותר (k הוא מספר קודי השגיאה השונים). 
# השימוש ב Threads מקביל את התהליך ולכן מקצר את זמן הריצה, אך הסיבוכיות האסימפטוטית לא משתנה. 

In [11]:
#  2. סיבוכיות זיכרון:

# O(chunk_size + k)
# O(chunk_size = 100000): אחסון חלקי הנתונים בזיכרון.
# O(k): אחסון ספירות השגיאות (k הוא מספר קודי השגיאה השונים).
# זכרון נשמר במקטעים, לכן הסיבוכיות תלויה בגודל המקטע. -->

In [14]:
# 1. ב.1.א
import pandas as pd

def perform_data_checks(df):
    print("מתחיל בדיקות נתונים...")

    try:
        pd.to_datetime(df['timestamp'], format='%d/%m/%Y %H:%M')
        print("בדיקה: פורמט התאריך תקין.")
    except ValueError:
        raise ValueError("שגיאה: פורמט התאריך בעמודת 'timestamp' אינו תקין.")

    duplicates = df.duplicated()
    if duplicates.any():
        print(f"אזהרה: נמצאו {duplicates.sum()} שורות כפולות.")
        # אפשרות להסיר כפילויות:
        # df = df.drop_duplicates(inplace=True)
    else:
        print("בדיקה: לא נמצאו כפילויות.")

    # בדיקה של ערכים חסרים
    if df.isnull().any().any():
        print("אזהרה: נמצאו ערכים חסרים:")
        print(df.isnull().sum())
        # אפשרות לטפל בערכים חסרים (ע"י הסרה)
        # df.dropna(inplace=True)
    else:
        print("בדיקה: לא נמצאו ערכים חסרים.")

    if not pd.api.types.is_numeric_dtype(df['value']):
        raise TypeError("שגיאה: עמודת 'value' חייבת להכיל נתונים מספריים.")
    else:
        print("בדיקה: סוג הנתונים בעמודת 'value' תקין.")

    print("בדיקות נתונים הסתיימו.")
    return df

try:
    time_series_df = pd.read_csv('time_series.csv')
    time_series_df = perform_data_checks(time_series_df)
    print("\nDataFrame לאחר בדיקות:")
    print(time_series_df.head())
except FileNotFoundError:
    print("שגיאה: קובץ time_series.csv לא נמצא.")
except ValueError as e:
    print(e)
except TypeError as e:
    print(e)

מתחיל בדיקות נתונים...
בדיקה: פורמט התאריך תקין.
אזהרה: נמצאו 88911 שורות כפולות.
אזהרה: נמצאו ערכים חסרים:
timestamp        0
value        99040
dtype: int64
שגיאה: עמודת 'value' חייבת להכיל נתונים מספריים.


In [23]:

import pandas as pd

def calc_hourly_avg(file, output_file):
    df = pd.read_csv(file, parse_dates=['timestamp'], dayfirst=True)
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['hour'] = df['timestamp'].dt.floor('h')
    hour_avg = df.groupby('hour')['value'].mean().reset_index()
    hour_avg['hour'] = hour_avg['hour'].apply(lambda x: x.strftime('%Y-%m-%d %H:00:00'))
    hour_avg.rename(columns={'hour': 'time_start', 'value': 'mean_value'}, inplace=True)
    hour_avg = hour_avg[['mean_value', 'time_start']]
    hour_avg.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Hourly averages saved to '{output_file}'")

file = 'time_series.csv'
hourly_avg_file = 'hourly_averages.csv'
calc_hourly_avg(file, hourly_avg_file)
hourly_avg = pd.read_csv(hourly_avg_file)
print(hourly_avg.head())

# סיבוכיות זמן - O(m log h)
# כי עובר על כל השורות ומקבץ לפי השעות

# סיבוכיות מקום - O(m)
# כי שומר את כל השורות בזיכרון
# סיבוכיות מקום לקובץ של השעות- O(h)
# כי שומר לפי מספר השעות בזיכרון

Hourly averages saved to 'hourly_averages.csv'
   mean_value           time_start
0   50.562894  2025-06-01 00:00:00
1   49.939803  2025-06-01 01:00:00
2   49.457213  2025-06-01 02:00:00
3   50.181573  2025-06-01 03:00:00
4   48.611496  2025-06-01 04:00:00


In [21]:
# 1.  ב.1.ב
# חלוקה לפי ימים מבחינה לוגית

def calculate_hourly_average_parts(file_path, output_file):
    all_hourly_averages = []
    try:
        df = pd.read_csv(file_path, parse_dates=['timestamp'], dayfirst=True)
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df['date'] = df['timestamp'].dt.date
        unique_dates = sorted(df['date'].unique())

        for date in unique_dates:
            daily_data = df[df['date'] == date].copy()
            daily_data['hour'] = daily_data['timestamp'].dt.floor('h')
            hourly_avg = daily_data.groupby('hour')['value'].mean().reset_index()
            hourly_avg['hour'] = hourly_avg['hour'].apply(lambda x: x.strftime('%Y-%m-%d %H:00:00'))
            hourly_avg.rename(columns={'hour': 'time_start', 'value': 'mean_value'}, inplace=True)
            all_hourly_averages.append(hourly_avg[['mean_value', 'time_start']])

        if all_hourly_averages:
            final_hourly_averages = pd.concat(all_hourly_averages).sort_values(by='time_start').reset_index(drop=True)
            final_hourly_averages.to_csv(output_file, index=False, encoding='utf-8')
            print(f"Hourly averages (processed daily) saved to '{output_file}'")
        else:
            print("לא נמצאו נתונים לעיבוד.")

    except FileNotFoundError:
        print(f"שגיאה: קובץ לא נמצא: {file_path}")
    except Exception as e:
        print(f"שגיאה כללית: {e}")

file = 'time_series.csv'
output_file_simple = 'hourly_averages_1.csv'
calculate_hourly_average_parts(file, output_file_simple)

hourly_avg_simple = pd.read_csv(output_file_simple)
print(hourly_avg_simple.head())


# סיבוכיות זמן - O(m log h)
# כי עובר על כל השורות ומקבץ לפי השעות

# סיבוכיות מקום - O(m)
# כי טוען בתחילה את כל הקובץ
# מעבד כל פעם רק O(h_day)
# לכל יום את כמות השעות באותו יום
# שומר בסוף קובץ בגודל O(h) כמות השעות

Hourly averages (processed daily) saved to 'hourly_averages_1.csv'
   mean_value           time_start
0   50.562894  2025-06-01 00:00:00
1   49.939803  2025-06-01 01:00:00
2   49.457213  2025-06-01 02:00:00
3   50.181573  2025-06-01 03:00:00
4   48.611496  2025-06-01 04:00:00


In [22]:
# חלוקה פיזית
import pandas as pd
import os

def calculate_hourly_average(input_file, output_dir, final_output_file):
    try:
        os.makedirs(output_dir, exist_ok=True)
        df = pd.read_csv(input_file, parse_dates=['timestamp'], dayfirst=True)
        df.sort_values(by='timestamp', inplace=True) 

        grouped = df.groupby(df['timestamp'].dt.date)

        all_hourly_averages = []
        for date, daily_data in grouped:
            daily_file = os.path.join(output_dir, f"daily_data_{date}.csv")
            daily_data[['timestamp', 'value']].to_csv(daily_file, index=False, encoding='utf-8')
            print(f"קובץ יומי נוצר: {daily_file}")

            daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)
            daily_df['value'] = pd.to_numeric(daily_df['value'], errors='coerce')
            daily_df['hour'] = daily_df['timestamp'].dt.floor('h')
            hourly_avg = daily_df.groupby('hour')['value'].mean().reset_index()
            hourly_avg['hour'] = hourly_avg['hour'].apply(lambda x: x.strftime('%Y-%m-%d %H:00:00'))
            hourly_avg.rename(columns={'hour': 'time_start', 'value': 'mean_value'}, inplace=True)
            all_hourly_averages.append(hourly_avg[['mean_value', 'time_start']])
            os.remove(daily_file)

        if all_hourly_averages:
            final_hourly_averages = pd.concat(all_hourly_averages).sort_values(by='time_start').reset_index(drop=True)
            final_hourly_averages.to_csv(final_output_file, index=False, encoding='utf-8')
            print(f"ממוצעים שעתיים נשמרו ב: {final_output_file}")
        else:
            print("לא נמצאו נתונים לעיבוד.")

    except FileNotFoundError:
        print(f"שגיאה: קובץ לא נמצא: {input_file}")
    except Exception as e:
        print(f"שגיאה כללית: {e}")

input_file = 'time_series.csv'
output_directory = 'daily_data_sorted'
final_output_file = 'hourly_averages_2.csv'

calculate_hourly_average(input_file, output_directory, final_output_file)

hourly_avg_final = pd.read_csv(final_output_file)
print(hourly_avg_final.head())

# סיבוכיות זמן - O(m) - טעינת הקובץ
# O(m log h) - מיון לפי תאריך
# O(m log d) - חלוקה לפי ימים
# O(h_day log h) - חישוב ממוצעים לפי שעות
# O(h_day) - שמירה לקובץ

# סיבוכיות מקום - O(m)
# כי טוען בתחילה את כל הקובץ
# מעבד כל פעם רק O(h_day)
# לכל יום את כמות השעות באותו יום
# שומר בסוף קובץ בגודל O(h) כמות השעות

קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-01.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-02.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-03.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-04.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-05.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-06.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-07.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-08.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-09.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-10.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-11.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-12.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-13.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-14.csv


C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)
C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)


קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-15.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-16.csv


C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)
C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)


קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-17.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-18.csv


...

C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)


קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-28.csv
קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-29.csv


C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)
C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)


קובץ יומי נוצר: daily_data_sorted\daily_data_2025-06-30.csv
ממוצעים שעתיים נשמרו ב: hourly_averages_2.csv
   mean_value           time_start
0   50.562894  2025-01-06 00:00:00
1   49.939803  2025-01-06 01:00:00
2   49.457213  2025-01-06 02:00:00
3   50.181573  2025-01-06 03:00:00
4   48.611496  2025-01-06 04:00:00


C:\Users\user1\AppData\Local\Temp\ipykernel_39384\1602750990.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  daily_df = pd.read_csv(daily_file, parse_dates=['timestamp'], dayfirst=True)


In [ ]:
# ב.3
# אם הנתונים מגיעים בזרימה

# 1. נשמור במילון -  hashmap
# סכום ומונה לכל שעה 
# 2. נעדכן סכום ומונה לשעה המתאימה
# 3. נחשב ממוצע לפי חלוקה של הסכום למונה
# 4. אפשר לעדכן בזמן אמת או במרווחים של זמן

hourly_sums = {}
hourly_counts = {}

def update_hourly_averages(timestamp, value):
    hour = timestamp.floor('H')
    if hour not in hourly_sums:
        hourly_sums[hour] = 0
        hourly_counts[hour] = 0
    hourly_sums[hour] += value
    hourly_counts[hour] += 1
    return hour, hourly_sums[hour] / hourly_counts[hour]



In [18]:
# 1. ב.4
import pandas as pd

def load_data(file_path):

    if file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    elif file_path.endswith('.parquet'):
        data = pd.read_parquet(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Parquet file.")
    return data

def calculate_hourly_averages(data):
    
    data['timestamp'] = pd.to_datetime(data['timestamp'], dayfirst=True)
    data['value'] = pd.to_numeric(data['value'], errors='coerce')
    
    data['hour'] = data['timestamp'].dt.floor('H')
    
    hourly_averages = data.groupby('hour')['value'].mean().reset_index()
    hourly_averages.rename(columns={'hour': 'Timestamp', 'value': 'mean_value'}, inplace=True)
    
    return hourly_averages

def save_results(data, output_file):
    print(data.head())
    data.to_csv(output_file, index=False)

if __name__ == "__main__":
    input_file = "time_series_.parquet"  
    output_file = "hourly_averages_3.csv"
    
    data = load_data(input_file)
    
    hourly_averages = calculate_hourly_averages(data)
    
    save_results(hourly_averages, output_file)
    print(f"Hourly averages saved to {output_file}")


# יתרונות בשימוש ב- parquet:
# 1. גודל קובץ קטן יותר - parquet דוחס את הנתונים בצורה טובה יותר.
# 2. מהירות קריאה - parquet מיועד לקריאה מהירה יותר של נתונים גדולים.
# 3. תמיכה בנתונים לא מסודרים - parquet תומך בנתונים לא מסודרים ובסכמות שונות.
# 4. תמיכה ב- predicate pushdown - parquet מאפשר סינון נתונים בצורה טובה יותר.
# 5. תמיכה ב- columnar storage - parquet שומר את הנתונים בצורה עמודתית, מה שמאפשר קריאה מהירה יותר של נתונים.



            Timestamp  mean_value
0 2025-06-01 00:00:00   50.562894
1 2025-06-01 01:00:00   49.939803
2 2025-06-01 02:00:00   49.457213
3 2025-06-01 03:00:00   50.181573
4 2025-06-01 04:00:00   48.611496
Hourly averages saved to hourly_averages_3.csv
